<a href="https://colab.research.google.com/github/jyotidabass/Integration-with-Other-Frameworks-Using-Hugging-Face-in-conjunction-with-other-frameworks-/blob/main/Integration_with_Other_Frameworks_Using_Hugging_Face_in_conjunction_with_other_frameworks_like_PyTorch_and_TensorFlow_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hugging Face with PyTorch**

In [4]:
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Define a PyTorch dataset class
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        # Preprocess text using Hugging Face tokenizer
        inputs = tokenizer(text, return_tensors='pt')

        # Get BERT embeddings
        outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

        # Return BERT embeddings and labels
        return outputs.last_hidden_state[:, 0, :], labels

    def __len__(self):
        return len(self.texts)

# Create a sample dataset
texts = ['This is a sample text.', 'This is another sample text.']
labels = [0, 1]

dataset = MyDataset(texts, labels)

# Create a PyTorch data loader
batch_size = 32  # Though set to 32, it will process 2 samples at a time in this case
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Train a PyTorch model using Hugging Face BERT embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Renamed to 'classifier' to avoid overwriting the BERT 'model'
classifier = torch.nn.Linear(768, 2)  # 768 is the dimensionality of BERT embeddings
classifier.to(device) # Send to the device

criterion = torch.nn.CrossEntropyLoss()
# Updated optimizer to use classifier's parameters
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-5)

for epoch in range(5):
    for batch in data_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass - pass only embeddings to the linear layer, remove extra dimension
        outputs = classifier(inputs.squeeze(1))
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update model parameters
        optimizer.step()

        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.6920924782752991
Epoch 2, Loss: 0.6914633512496948
Epoch 3, Loss: 0.6908381581306458
Epoch 4, Loss: 0.6902168989181519
Epoch 5, Loss: 0.6895996332168579


# **Hugging Face with TensorFlow**

In [6]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = TFBertModel.from_pretrained('bert-base-uncased')  # Renamed to avoid confusion

# Define a TensorFlow dataset class
class MyDataset:  # Inherit from object instead of tf.data.Dataset
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

        # Create a tf.data.Dataset from the text and labels
        self.dataset = tf.data.Dataset.from_tensor_slices((self.texts, self.labels))

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        # Preprocess text using Hugging Face tokenizer
        inputs = tokenizer(text, return_tensors='tf')

        # Get BERT embeddings
        outputs = model_bert(inputs['input_ids'], attention_mask=inputs['attention_mask'])

        # Return BERT embeddings and labels
        return outputs.last_hidden_state[:, 0, :], labels

    def __len__(self):
        return len(self.texts)

    def get_dataset(self):
        """Returns the tf.data.Dataset object."""
        return self.dataset


# Create a sample dataset
texts = ['This is a sample text.', 'This is another sample text.']
labels = [0, 1]

dataset = MyDataset(texts, labels)

# Create a TensorFlow data loader using the dataset's get_dataset method
batch_size = 32
data_loader = dataset.get_dataset().batch(batch_size)  # Modified

# Train a TensorFlow model using Hugging Face BERT embeddings
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(768,)),  # Input layer for BERT embeddings
    tf.keras.layers.Dense(2, activation='softmax')  # Output layer with 2 classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Training loop
for epoch in range(5):
    for batch in data_loader:
        inputs, labels = batch

        # Preprocess inputs using Hugging Face tokenizer
        inputs = tokenizer(inputs.numpy().astype(str).tolist(), return_tensors='tf', padding=True, truncation=True)

        # Get BERT embeddings using the TFBertModel (model_bert)
        with tf.GradientTape() as tape:
            embeddings = model_bert(inputs['input_ids'], attention_mask=inputs['attention_mask']).last_hidden_state[:, 0, :]

            # Get the classification output using the Sequential model (model)
            outputs = model(embeddings)
            loss = tf.keras.losses.sparse_categorical_crossentropy(labels, outputs)

        # Calculate gradients
        gradients = tape.gradient(loss, model.trainable_variables)

        # Update model weights
        model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        print(f'Epoch {epoch + 1}, Loss: {loss.numpy().mean()}, Accuracy: {tf.keras.metrics.sparse_categorical_accuracy(labels, outputs).numpy().mean()}')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1, Loss: 0.7286123037338257, Accuracy: 0.5
Epoch 2, Loss: 0.6961890459060669, Accuracy: 0.5
Epoch 3, Loss: 0.6651514768600464, Accuracy: 1.0
Epoch 4, Loss: 0.6350632905960083, Accuracy: 1.0
Epoch 5, Loss: 0.6060135364532471, Accuracy: 1.0
